In [1]:
import os

import numpy as np
import pandas as pd

from analysis import isovisc, utilities, common
reader = isovisc.data.reader
from everest.h5anchor import Fetch, Scope
from everest.window import Canvas, plot, raster, DataChannel as Channel, get_cmap
%matplotlib inline

In [2]:
inputs, initials, finals, averages = isovisc.data.get_summary_frames()

In [3]:
rasters = isovisc.data.get_rasters()

In [ ]:
from PIL import Image
class Rasters(dict):
    def __init__(self, path):
        super().__init__(
            (fname[:-4], os.path.join(path, fname))
                for fname in os.listdir(path)
            )
    def __getitem__(self, key):
        return Image.open(super().__getitem__(key))

In [ ]:
initrasts = Rasters(rasters[0])

In [ ]:
inputs.index

In [ ]:
len(initrasts)

In [ ]:
initrasts['aeghaugrusf-aotweengoasf'].resize((512, 512))

In [ ]:
import PIL

In [ ]:
os.path.join(rasters[0], os.listdir(rasters[0])[0])

In [ ]:
img = Image.open('/home/morpheus/workspace/mount/cache/rasters/isovisc/initial/tzatsor-uozhiidiesw.png')

In [ ]:
img.resize((256, 256))

In [ ]:
def get_maxres_indices(inputs):
    '''Get hashIDs of only those models which equal the max resolution available for those parameters.'''
    effkeys = [key for key in inputs.columns if not key in {'res', 'temperatureField'}]
    mygrp = inputs.groupby(effkeys)
    maxres = mygrp['res'].apply(max)
    reind = inputs.reset_index().set_index(effkeys)
    reind['maxres'] = maxres
    selection = reind.loc[reind['res'] == reind['maxres']]['hashID']
    return selection

def linear2_fineres(alphaexp):
    return max(8, round(2 ** alphaexp / 4) * 4)

import math
def linear_regression(x, y, log = False):
    if log:
        x, y = np.log(x), np.log(y)
    linreg = LinearRegression().fit(
        xlin := x.reshape(-1, 1),
        ylin := y,
        )
    pred = linreg.predict(xlin)
    r2 = r2_score(ylin, pred)
    slope, intercept = linreg.coef_, linreg.intercept_
    if log:
        predictor = lambda x: math.e ** linreg.predict(np.log(x).reshape(-1, 1))
        return predictor, math.e ** intercept, slope, r2
    predictor = lambda x: linreg.predict(x.reshape(-1, 1))
    return predictor, slope, intercept, r2

def jarvis_geom(f):
    f = 0.999 if f >= 1 else f
    return -np.log(f) / ((1 - f) * (1 + f ** (-3 / 4)) ** (4 / 3))

def colour(val, vals, cmap):
    return cmap((val - min(vals)) / (max(vals) - min(vals)))

In [ ]:
canvas = Canvas(size = (12, 12), fill = 'black', colour = 'white')
ax = canvas.make_ax(title = 'Isoviscous convection:\nNusselt number as a function of buoyancy')
ax.scatter(
    x := Channel(inputs['alpha'], label = 'alpha', lims = (None, None), capped = (True, None), islog = True),
    y := Channel(averages['Nu'], label = 'Nu', lims = (0, None), capped = (True, None), log = True),
    30 * (2 * inputs['f'] - 1) + 1,
    c = inputs['H'],
    marker = "_",
    cmap = 'plasma',
    )
canvas.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

selection = inputs.loc[get_maxres_indices(inputs)]
selection = dict(list(selection.groupby(['H', 'aspect'])))[0, 1]
selection = selection.loc[selection['alpha'] >= 4]
# selection = (selinit := initials.loc[selection.index]).loc[selinit['psi_av'] > -2000]
selection = selection.loc[(selinit := initials.loc[selection.index]).loc[selinit['theta_range'] > 1].index]
selection = selection.loc[initials.loc[selection.index]['theta_range'] > 1]
selection = selection.loc[initials.loc[selection.index]['Nu_min'] < 2]
selection = selection.loc[
    (selinps := inputs.loc[selection.index]).loc[selinps['res'] == selinps['alpha'].apply(linear2_fineres)].index
    ]
selection = selection.index

linreg = LinearRegression().fit(
    x := inputs.loc[selection]['alpha'].to_numpy().reshape(-1, 1),
    y := averages.loc[selection]['Nu'].apply(np.log10),
    )
pred = linreg.predict(x)
r2 = r2_score(y, pred)
slope, intercept = linreg.coef_, linreg.intercept_

canvas = Canvas(size = (9, 9), fill = 'black', colour = 'white')

title = '''
Isoviscous convection:
Nusselt number as a function of buoyancy
{aspect = 1, H = 0}
'''

ax = canvas.make_ax(title = title)
ax.scatter(
    Channel(inputs.loc[selection]['alpha'], label = 'log10(alpha)', lims = (4, 7.5), capped = (True, None)),
    Channel(np.log10(averages.loc[selection]['Nu']), label = 'log10(Nu)', lims = (0.6, 1.8), capped = (True, None)),
    20 * (2 * inputs.loc[selection]['f'] - 1) + 1,
    c = initials.loc[selection]['psi_av'],
    marker = "_",
    cmap = 'viridis',
    )
ax.line(x, pred)
# slopeformat = "{:.2e}".format(slope[0])
label = f"y = {round(float(slope), 4)} x + {round(intercept, 4)}\nr2 = {round(r2, 4)}"
ax.annotate(
    *list(zip(x, pred))[round(len(pred) / 2)],
    label,
    points = (-45, 45),
    arrowProps = {'arrowstyle': '->', 'color': 'white'},
    color = 'white'
    )
canvas.show()

In [ ]:
jarv = inputs['f'].apply(jarvis_geom)

In [ ]:
canvas = Canvas(size = (12, 12), colour = 'white', fill = 'black')
ax = canvas.make_ax()
cmap = get_cmap('viridis')
alphas = list(set(inputs.loc[selection]['alpha']))

for alpha, subfrm in inputs.groupby('alpha'):
    subsel = sorted(set.intersection(set(selection), set(subfrm.index)))
    if not len(subsel) > 1:
#         alphas.remove(alpha)
        continue
    fs = inputs.loc[subsel]['f'].sort_values()
    ax.line(
        fs,
        Channel(averages.loc[fs.index]['Nu'] / jarv.loc[fs.index], capped = (True, None)),
        c = colour(alpha, alphas, cmap),
        )
# ax.props.legend.set_handles_labels(
#     (row[0] for row in ax.collections),
#     alphas,
#     )
canvas.show()

In [ ]:
canvas = Canvas(size = (12, 6), colour = 'white', fill = 'black', dpi = 100)
ax = canvas.make_ax(
    title = 'Power-law behaviour of isoviscous convection\n{aspect = 1, H = 0}',
    )
ax.scatter(
    Channel(
        x := 10 ** inputs.loc[selection]['alpha'].to_numpy(),
        label = 'alpha',
        ),
    Channel(
        y := averages.loc[selection]['Nu'],
        label = 'Nu',
#         lims = (1, 2.2),
        capped = (True, None),
        ),
    c = inputs.loc[selection]['f']
    )

predictor, coefficient, exponent, r2 = linear_regression(x, y, log = True)

ax.line(
    x := np.arange(1, max(x), 1e3),
    y := predictor(x),
    color = 'white'
    )
# slopeformat = "{:.2e}".format(slope[0])
stco, stex, str2 = (str(arg) for arg in (round(coefficient, 4), round(float(exponent), 4), round(r2, 4)))
label = r"$Nu = " + stco + r"*Ra^{" + stex + r"}$" + '\n' + r"$r^{2} = " + str2 + r"$"
# label = f"Nu_j = {} Ra ^ {}\nr2 = "
ax.annotate(
    *list(zip(x, y))[round(len(y) / 2) - 1500],
    label,
    points = (0, -60),
    arrowProps = {'arrowstyle': '->', 'color': 'white'},
    color = 'white',
    fontsize = 12,
    )

canvas.show()

In [ ]:
canvas = Canvas(size = (12, 6), colour = 'white', fill = 'black', dpi = 100)
ax = canvas.make_ax(
    title = 'Power-law behaviour of isoviscous convection\n{aspect = 1, H = 0}',
    )
ax.scatter(
    Channel(
        x := 10 ** inputs.loc[selection]['alpha'].to_numpy(),
        label = 'alpha',
        ),
    Channel(
        y := averages.loc[selection]['Nu'] / jarv.loc[selection],
        label = 'Nu_Jarvis',
#         lims = (1, 2.2),
        capped = (True, None),
        ),
    c = inputs.loc[selection]['f']
    )

predictor, coefficient, exponent, r2 = linear_regression(x, y, log = True)

ax.line(
    x := np.arange(1, max(x), 1e3),
    y := predictor(x),
    color = 'white'
    )
# slopeformat = "{:.2e}".format(slope[0])
stco, stex, str2 = (str(arg) for arg in (round(coefficient, 4), round(float(exponent), 4), round(r2, 4)))
label = r"$Nu = " + stco + r"*Ra^{" + stex + r"}$" + '\n' + r"$r^{2} = " + str2 + r"$"
# label = f"Nu_j = {} Ra ^ {}\nr2 = "
ax.annotate(
    *list(zip(x, y))[round(len(y) / 2) - 1500],
    label,
    points = (0, -60),
    arrowProps = {'arrowstyle': '->', 'color': 'white'},
    color = 'white',
    fontsize = 12,
    )

canvas.show()